### Spark Moive Recommendation
In this notebook, I used an Alternating Least Squares (ALS) algorithm with Spark APIs to predict the ratings for the movies in [MovieLens small dataset](https://grouplens.org/datasets/movielens/latest/) which updated in 09/2018.

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math

In [3]:
import os
os.environ["PYSPARK_PYTHON"] = "python3"

## Part1: Data ETL and Data Exploration

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("moive analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [6]:
movies = spark.read.load("/FileStore/tables/movies.csv", format='csv', header = True)
ratings = spark.read.load("/FileStore/tables/ratings.csv", format='csv', header = True)
links = spark.read.load("/FileStore/tables/links.csv", format='csv', header = True)
tags = spark.read.load("/FileStore/tables/tags.csv", format='csv', header = True)

In [7]:
movies.show(5)

+-------+--------------------+--------------------+
movieId| title| genres|
+-------+--------------------+--------------------+
 1| Toy Story (1995)|Adventure|Animati...|
 2| Jumanji (1995)|Adventure|Childre...|
 3|Grumpier Old Men ...| Comedy|Romance|
 4|Waiting to Exhale...|Comedy|Drama|Romance|
 5|Father of the Bri...| Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows

In [8]:
ratings.show(5)

+------+-------+------+---------+
userId|movieId|rating|timestamp|
+------+-------+------+---------+
 1| 1| 4.0|964982703|
 1| 3| 4.0|964981247|
 1| 6| 4.0|964982224|
 1| 47| 5.0|964983815|
 1| 50| 5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows

In [9]:
links.show(5)

+-------+-------+------+
movieId| imdbId|tmdbId|
+-------+-------+------+
 1|0114709| 862|
 2|0113497| 8844|
 3|0113228| 15602|
 4|0114885| 31357|
 5|0113041| 11862|
+-------+-------+------+
only showing top 5 rows

In [10]:
tags.show(5)

+------+-------+---------------+----------+
userId|movieId| tag| timestamp|
+------+-------+---------------+----------+
 2| 60756| funny|1445714994|
 2| 60756|Highly quotable|1445714996|
 2| 60756| will ferrell|1445714992|
 2| 89774| Boxing story|1445715207|
 2| 89774| MMA|1445715200|
+------+-------+---------------+----------+
only showing top 5 rows

In [11]:
tmp1 = ratings.groupBy("userID").count().toPandas()['count'].min()
tmp2 = ratings.groupBy("movieId").count().toPandas()['count'].min()
print('For the users that rated movies and the movies that were rated:')
print('Minimum number of ratings per user is {}'.format(tmp1))
print('Minimum number of ratings per movie is {}'.format(tmp2))

For the users that rated movies and the movies that were rated:
Minimum number of ratings per user is 20
Minimum number of ratings per movie is 1

In [12]:
tmp1 = sum(ratings.groupBy("movieId").count().toPandas()['count'] == 1)
tmp2 = ratings.select('movieId').distinct().count()
print('{} out of {} movies are rated by only one user'.format(tmp1, tmp2))

3446 out of 9724 movies are rated by only one user

## Part 1: Spark SQL and OLAP

For this part, I mostly used spark sql to do the analysis and for some of the questions, I used both spark sql and regular sql to do the analysis.

### The number of Users

In [16]:
q1 = ratings.select('userID').distinct().count()
print('The number of Users is {}'.format(q1))

The number of Users is 610

### The number of Movies

In [18]:
q2 = movies.select('movieID').distinct().count()
print('The number of Movies is {}'.format(q2))

The number of Movies is 9742

### How many movies are rated by users? List movies not rated before

In [20]:
from pyspark.sql.functions import isnan, when, count, col, isnull

ratings.select([count(when(isnull('rating'), 'rating')).alias('ratingIsNull')]).show()
ratings.select('rating').filter('rating is null').show()
print('There are no null rating in the ratings table, which means that all the movies are in the list are rated before.')

+------------+
ratingIsNull|
+------------+
 0|
+------------+

+------+
rating|
+------+
+------+

There are no null rating in the ratings table, which means that all the movies are in the list are rated before.

In [21]:
q3 = ratings.select('movieID').distinct().count()
print('Since there are also {} movies in the movies dataset, which confirmed that all the movies are rated by users'.format(q3))

Since there are also 9724 movies in the movies dataset, which confirmed that all the movies are rated by users

### Movie Genres

There are total 19 genres in the list. And some of the movies are not listed with any genres.

In [24]:
import pyspark.sql.functions as f

q4_SepGen = movies.select(
        f.split("genres", "\|").alias("news"),
        f.posexplode(f.split("genres", "\|")).alias("pos", "genre")
    )

genres = q4_SepGen.select('genre').orderBy('genre').distinct()
genres.show()
genres.count()

+------------------+
 genre|
+------------------+
(no genres listed)|
 Action|
 Adventure|
 Animation|
 Children|
 Comedy|
 Crime|
 Documentary|
 Drama|
 Fantasy|
 Film-Noir|
 Horror|
 IMAX|
 Musical|
 Mystery|
 Romance|
 Sci-Fi|
 Thriller|
 War|
 Western|
+------------------+

 Out[ 25 ]: 20

### Movie for Each Category

Then I looked at the the number of movie for each genres to get a general understanding of the genre distribution. From the graph and chart, Drama has the most the number of movie (4361) and the second most is Comedy (3756). Notice there are overlaps when labeling the movies with genres. So Drama and Comedy are more oftenly labeled.

In [27]:
movies.createOrReplaceTempView("movies_df")

In [28]:
%sql
select a.new_genres, count(*) as Count from (select movieid, title, explode(split(genres, '[|]')) as new_genres from movies_df) as a group by a.new_genres order by a.new_genres;

new_genres,Count
(no genres listed),34
Action,1828
Adventure,1263
Animation,611
Children,664
Comedy,3756
Crime,1199
Documentary,440
Drama,4361
Fantasy,779


In [29]:
movies.select(
        "title",
        f.explode(f.split("genres", "\|")).alias("val")
    ).groupBy('val').count().orderBy('count').show()

+------------------+-----+
 val|count|
+------------------+-----+
(no genres listed)| 34|
 Film-Noir| 87|
 IMAX| 158|
 Western| 167|
 Musical| 334|
 War| 382|
 Documentary| 440|
 Mystery| 573|
 Animation| 611|
 Children| 664|
 Fantasy| 779|
 Horror| 978|
 Sci-Fi| 980|
 Crime| 1199|
 Adventure| 1263|
 Romance| 1596|
 Action| 1828|
 Thriller| 1894|
 Comedy| 3756|
 Drama| 4361|
+------------------+-----+

In [30]:
%sql 
SELECT new_genres as Category, collect_list(title) AS Movie_list FROM (select movieid, title, explode(split(genres, '[|]')) as new_genres from movies_df) as a group by new_genres order by Category;

Category Movie_list (no genres listed) List(La cravate (1957), Ben-hur (2016), Pirates of the Caribbean: Dead Men Tell No Tales (2017), Superfast! (2015), Let It Be Me (1995), Trevor Noah: African American (2013), Guardians (2016), Green Room (2015), The Brand New Testament (2015), Hyena Road, The Adventures of Sherlock Holmes and Doctor Watson, A Cosmic Christmas (1977), Grease Live (2016), Noin 7 veljestä (1968), Paterson, Ali Wong: Baby Cobra (2016), A Midsummer Night's Dream (2016), The Forbidden Dance (1990), Ethel & Ernest (2016), Whiplash (2013), The OA, Lemonade (2016), Cosmos, Maria Bamford: Old Baby, Death Note: Desu nôto (2006–2007), Generation Iron 2, T2 3-D: Battle Across Time (1996), The Godfather Trilogy: 1972-1990 (1992), The Adventures of Sherlock Holmes and Doctor Watson: The Hunt for the Tiger (1980), The Putin Interviews (2017), Black Mirror, Too Funny to Fail: The Life and Death of The Dana Carvey Show (2017), Serving in Silence: The Margarethe Cammermeyer Story (1995), A Christmas Story Live! (2017)) Action List(Heat (1995), Sudden Death (1995), GoldenEye (1995), Cutthroat Island (1995), Money Train (1995), Assassins (1995), Dead Presidents (1995), Mortal Kombat (1995), Lawnmower Man 2: Beyond Cyberspace (1996), From Dusk Till Dawn (1996), Fair Game (1995), Screamers (1995), Crossing Guard, The (1995), White Squall (1996), Nick of Time (1995), Broken Arrow (1996), Braveheart (1995), Rumble in the Bronx (Hont faan kui) (1995), Bad Boys (1995), Rob Roy (1995), Batman Forever (1995), Congo (1995), Desperado (1995), Die Hard: With a Vengeance (1995), First Knight (1995), Hackers (1995), Johnny Mnemonic (1995), Judge Dredd (1995), Mighty Morphin Power Rangers: The Movie (1995), Net, The (1995), Strange Days (1995), Under Siege 2: Dark Territory (1995), Waterworld (1995), Drop Zone (1994), French Kiss (1995), Hunted, The (1995), Star Wars: Episode IV - A New Hope (1977), New York Cop (Nyû Yôku no koppu) (1993), Natural Born Killers (1994), Outbreak (1995), Léon: The Professional (a.k.a. The Professional) (Léon) (1994), Quick and the Dead, The (1995), Specialist, The (1994), Stargate (1994), Tank Girl (1995), Tales from the Hood (1995), Virtuosity (1995), Clear and Present Danger (1994), Crow, The (1994), I Love Trouble (1994), Mask, The (1994), Naked Gun 33 1/3: The Final Insult (1994), River Wild, The (1994), Speed (1994), Timecop (1994), True Lies (1994), Bad Company (1995), Low Down Dirty Shame, A (1994), Faster Pussycat! Kill! Kill! (1965), Street Fighter (1994), Highlander III: The Sorcerer (a.k.a. Highlander: The Final Dimension) (1994), Beverly Hills Cop III (1994), Blown Away (1994), Cliffhanger (1993), Cowboy Way, The (1994), Demolition Man (1993), Getaway, The (1994), Hard Target (1993), Hot Shots! Part Deux (1993), In the Line of Fire (1993), Judgment Night (1993), Jurassic Park (1993), Last Action Hero (1993), Menace II Society (1993), Executive Decision (1996), Next Karate Kid, The (1994), No Escape (1994), Program, The (1993), Rising Sun (1993), RoboCop 3 (1993), Romper Stomper (1992), Shadow, The (1994), Blade Runner (1982), Striking Distance (1993), Super Mario Bros. (1993), Surviving the Game (1994), Terminal Velocity (1994), Three Musketeers, The (1993), Tombstone (1993), Pagemaster, The (1994), Terminator 2: Judgment Day (1991), Batman (1989), Love and a .45 (1994), Heavy Metal (1981), Hellraiser: Bloodline (1996), Courage Under Fire (1996), Mission: Impossible (1996), Dragonheart (1996), Bloodsport 2 (a.k.a. Bloodsport II: The Next Kumite) (1996), Operation Dumbo Drop (1995), Solo (1996), Substitute, The (1996), Quest, The (1996), Rock, The (1996), Twister (1996), Barb Wire (1996), Arrival, The (1996), Phantom, The (1996), Independence Day (a.k.a. ID4) (1996), Eraser (1996), Daylight (1996), Fled (1996), Chain Reaction (1996), Crow: City of Angels, The (1996), Escape from L.A. (1996), Supercop (Police Story 3: Supercop) (Jing cha gu shi III: Chao ji jing cha) (1992), Nothing to Lose (1994), 

In [31]:
movies.select(
        "title",
        f.explode(f.split("genres", "\|")).alias("Category")
    )\
    .groupBy('Category').agg(f.concat_ws(',', f.collect_list('title')).alias('Movie_List')).orderBy('Category').show()

+------------------+--------------------+
 Category| Movie_List|
+------------------+--------------------+
(no genres listed)|La cravate (1957)...|
 Action|Heat (1995),Sudde...|
 Adventure|Toy Story (1995),...|
 Animation|Toy Story (1995),...|
 Children|Toy Story (1995),...|
 Comedy|Toy Story (1995),...|
 Crime|Heat (1995),Casin...|
 Documentary|Nico Icon (1995),...|
 Drama|Waiting to Exhale...|
 Fantasy|Toy Story (1995),...|
 Film-Noir|Devil in a Blue D...|
 Horror|Dracula: Dead and...|
 IMAX|Apollo 13 (1995),...|
 Musical|Pocahontas (1995)...|
 Mystery|Copycat (1995),Ci...|
 Romance|Grumpier Old Men ...|
 Sci-Fi|Powder (1995),Cit...|
 Thriller|Heat (1995),Golde...|
 War|Richard III (1995...|
 Western|Desperado (1995),...|
+------------------+--------------------+

For Drama, Comedy and Thriller these top three genres, the graphs below shows that these three categories share a very common trend in the year distribution of movies.

In [33]:
#Drama
movies_genre = movies.select("title", f.explode(f.split("genres", "\|")).alias("val"))
display(movies_genre.filter(movies_genre.val == 'Drama').where('title rlike "\((.*)\)"').selectExpr("substring(title, length(title)-4, 4) as Year").groupBy('Year').count().orderBy('Year'))

Year,count
002),1
007),1
011),2
1915,1
1916,1
1919,1
1921,1
1923,2
1924,2
1925,2


In [34]:
#Comedy
display(movies_genre.filter(movies_genre.val == 'Comedy').where('title rlike "\((.*)\)"').selectExpr("substring(title, length(title)-4, 4) as Year").groupBy('Year').count().orderBy('Year'))

Year,count
014),1
1908,1
1916,1
1917,1
1919,1
1920,1
1921,1
1923,2
1924,2
1925,2


In [35]:
#Thriller
display(movies_genre.filter(movies_genre.val == 'Thriller').where('title rlike "\((.*)\)"').selectExpr("substring(title, length(title)-4, 4) as Year").groupBy('Year').count().orderBy('Year'))

Year,count
011),1
1924,1
1927,1
1931,1
1933,1
1934,1
1935,1
1936,3
1937,1
1938,2


## Part2: Spark ALS based approach for training model
Reference to Spark ML ALS model:

https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/4999972933037924/899848065201823/8135547933712821/latest.html

In this part, I used DataFrame-based API from pyspark.mllib to predict the ratings. After reading the rating data file, I first change all the column types into float and column names into corresponding names such as userId, movieId and ratings in order to do further model fitting. Then I split the data into training/validation/testing sets using a 6/2/2 ratio.

In [38]:
from pyspark.ml.recommendation import ALS
from pyspark.sql.types import *
from pyspark.ml.evaluation import RegressionEvaluator

In [39]:
movie_rating = sc.textFile("/FileStore/tables/ratings.csv")

In [40]:
header = movie_rating.take(1)[0]
rating_data = movie_rating.filter(lambda line: line!=header).map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()

In [41]:
# check three rows
rating_data.take(3)

Out[ 103 ]: [('1', '1', '4.0'), ('1', '3', '4.0'), ('1', '6', '4.0')]

In [42]:
rating_new = rating_data.toDF(["userId", "movieId", "rating"])
type(rating_new)
rating_new = rating_new.withColumn("userId", rating_new["userId"].cast(IntegerType())).withColumn("movieId", rating_new["movieId"].cast(IntegerType())).withColumn("rating", rating_new["rating"].cast(FloatType()))
train, validation, test = rating_new.randomSplit([0.6, 0.2, 0.2],seed = 7856)

Check that the range of rating is valid.

In [44]:
rating_new.select('rating').summary('min','max').show()

+-------+------+
summary|rating|
+-------+------+
 min| 0.5|
 max| 5.0|
+-------+------+

In [45]:
train.cache()

Out[ 106 ]: DataFrame[userId: int, movieId: int, rating: float]

In [46]:
validation.cache()

Out[ 107 ]: DataFrame[userId: int, movieId: int, rating: float]

In [47]:
test.cache()

Out[ 108 ]: DataFrame[userId: int, movieId: int, rating: float]

### ALS Model Selection and Evaluation

With the ALS model, we can use a grid search to find the optimal hyperparameters.

In [49]:
def train_ALS(train_data, validation_data, num_iters, reg_param, ranks):
    min_error = float('inf')
    best_rank = -1
    best_regularization = 0
    best_model = None
    for rank in ranks:
        for reg in reg_param:
            # write your approach to train ALS model
            als = ALS(rank = rank, maxIter = num_iters, regParam = reg, userCol = "userId", itemCol = "movieId", ratingCol = "rating", coldStartStrategy = "drop")
            model = als.fit(train_data)
            # make prediction
            predictions = model.transform(validation_data)
            # get the rating result
            evaluator = RegressionEvaluator(metricName = "rmse", labelCol = "rating", predictionCol = "prediction")
            # get the RMSE
            error = evaluator.evaluate(predictions)
            print ('{} latent factors and regularization = {}: validation RMSE is {}'.format(rank, reg, error))
            if error < min_error:
                min_error = error
                best_rank = rank
                best_regularization = reg
                best_model = model
    print ('\nThe best model has {} latent factors and regularization = {}'.format(best_rank, best_regularization))
    return best_model

In [50]:
num_iterations = 10
ranks = [6, 8, 10, 12, 14]
reg_params = [0.05, 0.1, 0.2, 0.4, 0.8]

import time
start_time = time.time()
final_model = train_ALS(train, validation, num_iterations, reg_params, ranks)

print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

6 latent factors and regularization = 0.05: validation RMSE is 0.9727829554077956
6 latent factors and regularization = 0.1: validation RMSE is 0.9110344873615535
6 latent factors and regularization = 0.2: validation RMSE is 0.8935236558255054
6 latent factors and regularization = 0.4: validation RMSE is 0.9604944848710424
6 latent factors and regularization = 0.8: validation RMSE is 1.1791125379973424
8 latent factors and regularization = 0.05: validation RMSE is 0.9856000619265894
8 latent factors and regularization = 0.1: validation RMSE is 0.9122359321022802
8 latent factors and regularization = 0.2: validation RMSE is 0.89053669515403
8 latent factors and regularization = 0.4: validation RMSE is 0.9605568950300075
8 latent factors and regularization = 0.8: validation RMSE is 1.179100584234415
10 latent factors and regularization = 0.05: validation RMSE is 1.002015330931951
10 latent factors and regularization = 0.1: validation RMSE is 0.9179815312156453
10 latent factors and regularization = 0.2: validation RMSE is 0.8931350570893857
10 latent factors and regularization = 0.4: validation RMSE is 0.9606799609594964
10 latent factors and regularization = 0.8: validation RMSE is 1.17911873048888
12 latent factors and regularization = 0.05: validation RMSE is 0.9979608296981989
12 latent factors and regularization = 0.1: validation RMSE is 0.9124799704257528
12 latent factors and regularization = 0.2: validation RMSE is 0.8913213883973399
12 latent factors and regularization = 0.4: validation RMSE is 0.9611158627498855
12 latent factors and regularization = 0.8: validation RMSE is 1.1791058895604922
14 latent factors and regularization = 0.05: validation RMSE is 1.0087033156838763
14 latent factors and regularization = 0.1: validation RMSE is 0.9181582548475572
14 latent factors and regularization = 0.2: validation RMSE is 0.8941292740247823
14 latent factors and regularization = 0.4: validation RMSE is 0.9609928572820123
14 latent factors and regularization = 0.8: validation RMSE is 1.1791013265184638

The best model has 8 latent factors and regularization = 0.2
Total Runtime: 999.31 seconds

In the model training part, I tried 5 different ranks, 5 different regularization parameters and for each combination, the iteration time is 10. It tooks approximately 15 minutes to finish the model selection. The best model has 8 latent factors and regularization = 0.2

In [52]:
def plot_learning_curve(iter_array, train, validation, reg, rank):
  learning_res = []
  
  for num_iters in iter_array:
    print ('Current iteration number is {}'.format(num_iters))
    als = ALS(rank = rank, maxIter = num_iters, regParam = reg, userCol = "userId", itemCol = "movieId", ratingCol = "rating", coldStartStrategy = "drop")
    model = als.fit(train)
    # make prediction
    predictions = model.transform(validation)
    # get the rating result
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
    # get the RMSE
    error = evaluator.evaluate(predictions)
    learning_res.append(error)
    
  learning_dict = {'iterations': iter_array, 'MSE': learning_res}
  learning_df = pd.DataFrame(learning_dict, columns = ['iterations', 'MSE'])
  
  return learning_df

In [53]:
iter_array = [1, 2, 5, 10, 15]
# write your function to plot the learning curve 
learning_pd = spark.createDataFrame(plot_learning_curve(iter_array, train, validation, 0.2, 8))
display(learning_pd)

iterations,MSE
1,3.297949460423895
2,1.0692652243503782
5,0.8927757007020671
10,0.8921939281580925
15,0.8922923817800666


With the best parameter combination, I plotted 5 different iteration times and we can see that when the iteration time reaches 5, the MSE is stable around 0.89 and does not seem to decrease a lot as the iteration time increases. So we can say the model starts to converge well under the best parameter setting after 5 iteration.

### Model testing on the test data
And finally, wite your code to make a prediction and check the testing error.

After having the best model parameters and iteration number, I combined train set and validation set to fit a final model for test data and the MSE for test is 0.8773 which is even slightly smaller than the training MSE.

In [57]:
best_rank = 8
best_reg = 0.2
best_iters = 10
best_data = train.union(validation)
best_als = ALS(rank = best_rank, maxIter = best_iters, regParam = best_reg, userCol = "userId", itemCol = "movieId", ratingCol = "rating", coldStartStrategy = "drop")
best_model = best_als.fit(best_data)
predictions = best_model.transform(test)
best_evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
test_rmse = best_evaluator.evaluate(predictions)
print("Root-mean-square test error = " + str(test_rmse))

Root-mean-square test error = 0.8772993309551398

In [58]:
# Generate top 10 movie recommendations for each user
userRecs = best_model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = best_model.recommendForAllItems(10)

For each user, the model generate top 10 movie recommendations based on the predicted non-scale rating.

In [60]:
display(userRecs)

userId,recommendations
471,"List(List(170355, 4.9130397), List(3379, 4.9130397), List(40491, 4.738419), List(141718, 4.7371697), List(8235, 4.712113), List(2295, 4.712113), List(6201, 4.712113), List(92494, 4.684959), List(102217, 4.684959), List(7025, 4.665813))"
463,"List(List(59018, 5.2449803), List(60943, 5.2449803), List(3379, 5.2185445), List(170355, 5.2185445), List(141718, 5.105129), List(33649, 5.0716324), List(102217, 5.0309906), List(92494, 5.0309906), List(132333, 4.986168), List(67618, 4.982903))"
496,"List(List(40491, 4.841043), List(170355, 4.720382), List(3379, 4.720382), List(59018, 4.5131845), List(60943, 4.5131845), List(141718, 4.485042), List(33649, 4.4279995), List(102217, 4.4063687), List(92494, 4.4063687), List(7025, 4.3697767))"
148,"List(List(33649, 4.586727), List(67618, 4.559931), List(25906, 4.5091085), List(77846, 4.5091085), List(170355, 4.4863043), List(3379, 4.4863043), List(141718, 4.4820514), List(25947, 4.4623427), List(3235, 4.4413705), List(6201, 4.3866887))"
540,"List(List(59018, 5.3416314), List(60943, 5.3416314), List(170355, 5.3229938), List(3379, 5.3229938), List(132333, 5.26339), List(33649, 5.194287), List(92494, 5.1762404), List(102217, 5.1762404), List(141718, 5.173386), List(67618, 5.052293))"
392,"List(List(59018, 4.845555), List(60943, 4.845555), List(170355, 4.8299727), List(3379, 4.8299727), List(141718, 4.6265483), List(92494, 4.6224127), List(102217, 4.6224127), List(3567, 4.543527), List(117531, 4.505463), List(26073, 4.505463))"
243,"List(List(67618, 5.749725), List(60943, 5.659555), List(59018, 5.659555), List(33649, 5.5353837), List(3086, 5.5003557), List(170355, 5.412776), List(3379, 5.412776), List(132333, 5.397106), List(179135, 5.391691), List(184245, 5.391691))"
31,"List(List(77846, 5.3806143), List(25906, 5.3806143), List(67618, 5.3560038), List(45503, 5.3146586), List(59018, 5.270289), List(60943, 5.270289), List(33649, 5.2424793), List(170355, 5.2382674), List(3379, 5.2382674), List(6201, 5.188435))"
516,"List(List(141718, 4.7361326), List(132333, 4.703351), List(60943, 4.6993937), List(59018, 4.6993937), List(170355, 4.6981626), List(3379, 4.6981626), List(2295, 4.6672277), List(8235, 4.6672277), List(6201, 4.6672277), List(92494, 4.610749))"
580,"List(List(60943, 4.9107347), List(59018, 4.9107347), List(67618, 4.805751), List(170355, 4.769459), List(3379, 4.769459), List(33649, 4.7525806), List(7071, 4.728626), List(184245, 4.728626), List(4256, 4.728626), List(179135, 4.728626))"


Similarly, for each movie, the model generate top 10 user recommendations based on the predicted non-scale rating.

In [62]:
display(movieRecs)

movieId,recommendations
1580,"List(List(53, 4.901009), List(543, 4.4891577), List(43, 4.45896), List(276, 4.3584614), List(452, 4.318888), List(93, 4.3109574), List(337, 4.228496), List(327, 4.22444), List(544, 4.208923), List(475, 4.203167))"
4900,"List(List(53, 3.6809359), List(543, 3.272677), List(544, 3.2094886), List(147, 3.2061398), List(327, 3.1950545), List(276, 3.1727414), List(492, 3.1591454), List(240, 3.144431), List(43, 3.1254668), List(243, 3.1050925))"
5300,"List(List(53, 4.43812), List(236, 4.210815), List(122, 4.103311), List(171, 4.090659), List(371, 4.065102), List(251, 4.064554), List(348, 4.0640435), List(515, 4.059045), List(375, 4.0357594), List(1, 4.0145555))"
6620,"List(List(236, 4.467574), List(296, 4.3655057), List(168, 4.1865997), List(53, 4.1425133), List(418, 4.1159544), List(122, 4.088429), List(191, 4.085511), List(187, 4.085014), List(409, 4.0835524), List(59, 4.0808053))"
7340,"List(List(53, 3.6756792), List(594, 3.5184994), List(452, 3.5081663), List(43, 3.4953072), List(558, 3.4918509), List(578, 3.4877572), List(543, 3.4364848), List(12, 3.4203956), List(584, 3.410496), List(169, 3.4103477))"
32460,"List(List(53, 5.3789606), List(452, 4.9062333), List(371, 4.87597), List(236, 4.8294253), List(523, 4.779258), List(515, 4.7773542), List(25, 4.77485), List(122, 4.7381196), List(93, 4.702956), List(276, 4.676432))"
54190,"List(List(53, 5.5012193), List(452, 4.9245996), List(276, 4.8788595), List(523, 4.782487), List(43, 4.774504), List(93, 4.766979), List(543, 4.7615056), List(371, 4.737882), List(475, 4.7251716), List(548, 4.7078753))"
471,"List(List(43, 4.480152), List(53, 4.391327), List(578, 4.341218), List(12, 4.3346825), List(171, 4.20584), List(250, 4.201273), List(236, 4.1834216), List(169, 4.173154), List(273, 4.168121), List(188, 4.1672177))"
1591,"List(List(53, 3.9177763), List(236, 3.6740012), List(371, 3.5855467), List(452, 3.560411), List(25, 3.537426), List(533, 3.5062275), List(441, 3.505924), List(548, 3.4779058), List(515, 3.474829), List(122, 3.4743176))"
140541,"List(List(393, 4.0988445), List(89, 3.811891), List(598, 3.7450526), List(138, 3.6988268), List(578, 3.674523), List(53, 3.6205173), List(548, 3.598946), List(30, 3.5650506), List(413, 3.5584984), List(97, 3.5173073))"
